# Mixture of Experts (MoE) Router — Unit 2 Assignment

**Author:** Sourabh S Hegde  
**SRN:** PES2UG23CS929  
**Class:** CSE 6K  
**Topic:** Advanced LLM Architecture with Groq API  
**Stack:** Python · Groq API · python-dotenv

---

## What I'm building

I wanted to explore how production AI systems route queries to specialized sub-models, so I built a **Smart Customer Support Router** based on the Mixture of Experts (MoE) pattern.

The idea is simple: instead of having one generic LLM handle everything, a lightweight **router** first classifies the user's intent, then forwards the query to the most appropriate **expert**:

- 🛠️ **Technical Expert** — handles bug reports and dev issues
- 💳 **Billing Expert** — deals with payment and subscription problems
- 🛍️ **Sales Expert** — answers product and pricing questions
- 📈 **Tool Use Expert** *(bonus)* — fetches real-time data
- 💬 **General Expert** — catches everything else

Each expert has its own persona, system prompt, and temperature — tuned for its specific job.

## Step 1 — Install Dependencies

In [16]:
%pip install groq python-dotenv --quiet

## Step 2 — API Key Setup

I keep my key in a `.env` file so I never accidentally commit it:

```
GROQ_API_KEY=your_groq_api_key_here
```

You can also uncomment the line below to set it inline (just don't push it to GitHub).

In [17]:
import os
from dotenv import load_dotenv
from groq import Groq

load_dotenv()

os.environ["GROQ_API_KEY"] = "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"

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Using llama-3.3-70b-versatile — mixtral-8x7b is deprecated as of mid-2024
BASE_MODEL = "llama-3.3-70b-versatile"

print("✅ Groq client ready.")

✅ Groq client ready.


## Step 3 — Expert Configurations

I define each expert as a dictionary entry with:
- A **system prompt** that gives the LLM a specific persona and instructions
- A **temperature** value (lower = more precise/deterministic, higher = more conversational)

Getting the system prompts right took a few iterations — the billing one especially needed an empathetic tone.

In [18]:
EXPERT_CONFIG = {
    "technical": {
        "system_prompt": (
            "You are a Senior Software Engineer specializing in debugging and technical support. "
            "Be precise and rigorous. Always identify the root cause before suggesting a fix. "
            "Provide corrected code snippets with inline comments explaining each change. "
            "Follow best practices and use accurate technical terminology. "
            "Format all code in fenced code blocks with the appropriate language tag."
        ),
        "temperature": 0.3,
    },
    "billing": {
        "system_prompt": (
            "You are a professional Billing and Accounts Specialist with a warm, reassuring tone. "
            "Always start by acknowledging the customer's concern before jumping into solutions. "
            "Walk them through refund procedures, subscription policies, and dispute resolution "
            "clearly and step-by-step. Never expose sensitive financial data, and escalate "
            "to a human agent whenever the situation warrants it."
        ),
        "temperature": 0.5,
    },
    "sales": {
        "system_prompt": (
            "You are an enthusiastic Sales Consultant who genuinely wants to help customers "
            "find the right plan or product for their needs. "
            "Highlight relevant features and value clearly, mention current promotions, "
            "and always end with a specific call-to-action. Be persuasive, not pushy."
        ),
        "temperature": 0.7,
    },
    "tool_use": {
        "system_prompt": (
            "You are a financial data assistant. Real-time data will be injected into your context. "
            "Use it to answer the user's question accurately. "
            "Always mention the data source and note that prices update frequently."
        ),
        "temperature": 0.2,
    },
    "general": {
        "system_prompt": (
            "You are a friendly, helpful general-purpose assistant. "
            "Keep your answers clear and conversational. "
            "Be concise unless the user asks for a detailed explanation."
        ),
        "temperature": 0.7,
    },
}

print(f"✅ {len(EXPERT_CONFIG)} experts loaded: {list(EXPERT_CONFIG.keys())}")

✅ 5 experts loaded: ['technical', 'billing', 'sales', 'tool_use', 'general']


## Step 4 — Bonus: Mock Crypto Price Tool

For the tool-use expert, I wrote a mock function that simulates a real-time price API.  
In a production setup you'd swap this for a live call to CoinGecko or Binance — the function signature stays the same.

In [19]:
def fetch_crypto_price(asset: str) -> dict:
    """
    Simulates a real-time crypto price lookup.

    In production, replace the mock_prices dict with a live API call, e.g.:
        import requests
        r = requests.get(f'https://api.coingecko.com/api/v3/simple/price?ids={asset}&vs_currencies=usd')
        return r.json()

    Args:
        asset: Name or ticker of the cryptocurrency (case-insensitive)

    Returns:
        dict with symbol, price_usd, change_24h — or an error key if not found
    """
    mock_prices = {
        "bitcoin":  {"symbol": "BTC", "price_usd": 68_420.50, "change_24h": "+2.3%"},
        "ethereum": {"symbol": "ETH", "price_usd":  3_512.75, "change_24h": "-0.8%"},
        "solana":   {"symbol": "SOL", "price_usd":    172.40, "change_24h": "+4.1%"},
    }

    query = asset.lower()
    for name, data in mock_prices.items():
        if name in query or data["symbol"].lower() in query:
            return data

    return {"error": f"No price data available for '{asset}'"}


# Sanity check
print(fetch_crypto_price("bitcoin"))
print(fetch_crypto_price("ETH"))

{'symbol': 'BTC', 'price_usd': 68420.5, 'change_24h': '+2.3%'}
{'symbol': 'ETH', 'price_usd': 3512.75, 'change_24h': '-0.8%'}


## Step 5 — The Router: `classify_intent()`

This is the core of the MoE pattern. I use a separate LLM call at `temperature=0` (fully deterministic)
to classify the incoming query into one of the five categories.  
Keeping `max_tokens=10` ensures we get just the category word — no extra fluff.

In [20]:
def classify_intent(user_input: str) -> str:
    """
    Routes a user query to the correct expert category.

    Categories:
        technical | billing | sales | tool_use | general

    Returns a single lowercase string. Falls back to 'general' if the
    model returns something unexpected.
    """
    router_system_prompt = """You are an intent classification engine for a multi-expert customer support system.
Classify the user message into EXACTLY ONE of the following categories:

- technical   : bug reports, code errors, crashes, API problems, developer how-to questions
- billing     : charges, refunds, invoices, subscriptions, payment disputes
- sales       : pricing inquiries, new product interest, plan upgrades, demo requests
- tool_use    : requests for live/real-time data such as crypto prices, stocks, or weather
- general     : greetings, small talk, or anything that doesn't fit the above

Rules:
1. Output ONLY the single category word in lowercase.
2. No punctuation, no explanation, no extra text whatsoever.
3. When in doubt, return: general"""

    resp = client.chat.completions.create(
        model=BASE_MODEL,
        messages=[
            {"role": "system", "content": router_system_prompt},
            {"role": "user",   "content": user_input},
        ],
        temperature=0,   # Deterministic — routing must be consistent
        max_tokens=10,   # Category is a single word; cap tokens to avoid noise
    )

    category = resp.choices[0].message.content.strip().lower()

    # Guard against unexpected outputs
    if category not in EXPERT_CONFIG:
        print(f"  ⚠️  Unexpected category '{category}' — defaulting to 'general'")
        category = "general"

    return category


# Quick tests
print(classify_intent("My Python script is throwing an IndexError"))
print(classify_intent("I was charged twice this month"))
print(classify_intent("What is the price of Ethereum?"))

technical
billing
tool_use


## Step 6 — The Orchestrator: `handle_request()`

This function ties everything together:
1. Calls `classify_intent()` to figure out which expert should respond
2. If `tool_use`: extracts the asset name, calls the mock tool, and injects the result into the expert's context
3. Otherwise: makes a standard LLM call using the appropriate expert's config

In [21]:
def handle_request(user_input: str) -> str:
    """
    MoE orchestrator — routes a query and calls the appropriate expert.

    Flow:
        user_input → classify_intent() → category
            if tool_use  → fetch live data → tool_use expert
            else         → relevant expert LLM call
    """
    print(f"\n{'='*60}")
    print(f"  📨 User Input  : {user_input}")

    # ── 1. Classify ──────────────────────────────────────────────
    category = classify_intent(user_input)
    print(f"  🔀 Routed To   : [{category.upper()} EXPERT]")
    print(f"{'='*60}")

    # ── 2a. Tool-use branch (bonus) ──────────────────────────────
    if category == "tool_use":
        # Use a focused LLM call to pull out just the asset name
        asset_resp = client.chat.completions.create(
            model=BASE_MODEL,
            messages=[{
                "role": "user",
                "content": (
                    f"Extract only the cryptocurrency or asset name from the query below. "
                    f"Return ONLY the asset name (e.g., 'bitcoin', 'ethereum', 'solana'). "
                    f"Query: '{user_input}'"
                )
            }],
            temperature=0,
            max_tokens=10,
        )
        asset_name = asset_resp.choices[0].message.content.strip().lower()

        # Fetch mock price data
        price_data = fetch_crypto_price(asset_name)
        print(f"  🔧 Tool Result : fetch_crypto_price('{asset_name}') → {price_data}")

        if "error" in price_data:
            return f"Sorry, I couldn't find price data for '{asset_name}'."

        # Inject tool result into the expert's context
        augmented_context = (
            f"Live market data for {price_data['symbol']}:\n"
            f"  Current Price : ${price_data['price_usd']:,.2f} USD\n"
            f"  24h Change    : {price_data['change_24h']}\n"
            f"  Data Source   : Mock Market Feed (simulated)\n\n"
            f"User question: {user_input}"
        )
        result = client.chat.completions.create(
            model=BASE_MODEL,
            messages=[
                {"role": "system", "content": EXPERT_CONFIG["tool_use"]["system_prompt"]},
                {"role": "user",   "content": augmented_context},
            ],
            temperature=EXPERT_CONFIG["tool_use"]["temperature"],
            max_tokens=300,
        )
        return result.choices[0].message.content

    # ── 2b. Standard expert branch ───────────────────────────────
    expert = EXPERT_CONFIG[category]
    result = client.chat.completions.create(
        model=BASE_MODEL,
        messages=[
            {"role": "system", "content": expert["system_prompt"]},
            {"role": "user",   "content": user_input},
        ],
        temperature=expert["temperature"],
        max_tokens=512,
    )
    return result.choices[0].message.content


print("✅ Orchestrator ready.")

✅ Orchestrator ready.


## Step 7 — End-to-End Demo

Running all five routing paths to verify everything works together.

In [22]:
test_queries = [
    "I'm getting a KeyError when accessing a dict inside a list of dicts.",  # → technical
    "I got charged twice for my subscription last week.",                     # → billing
    "Can you tell me what's included in the Pro plan?",                       # → sales
    "What's the current price of Bitcoin?",                                   # → tool_use (bonus)
    "Hey, what can you help me with?",                                        # → general
]

for query in test_queries:
    answer = handle_request(query)
    print(f"\n💬 Response:\n{answer}")
    print("-" * 60)


  📨 User Input  : I'm getting a KeyError when accessing a dict inside a list of dicts.
  🔀 Routed To   : [TECHNICAL EXPERT]

💬 Response:
To accurately diagnose and resolve the `KeyError`, we need to understand the structure of your data and the code that's causing the error. However, I can provide a general approach to troubleshooting and fixing this issue.

### Understanding the Error
A `KeyError` occurs when you try to access a key in a dictionary that does not exist. When dealing with a list of dictionaries, this could mean that the key you're trying to access is not present in one or more of the dictionaries in the list.

### Example Scenario
Let's consider an example where we have a list of dictionaries representing users, and each dictionary should have a 'name' and an 'age'. However, one of the dictionaries is missing the 'age' key.

```python
users = [
    {'name': 'John', 'age': 30},
    {'name': 'Jane'},  # Missing 'age' key
    {'name': 'Bob', 'age': 40}
]

# Trying to acce

## Step 8 — Interactive Mode

Type any query and watch the router decide which expert handles it. Type `exit` to quit.

In [23]:
print("🤖 MoE Customer Support Router — Interactive Mode")
print("Type 'exit' or 'quit' to stop.\n")

while True:
    user_input = input("You: ").strip()
    if not user_input or user_input.lower() in ("exit", "quit"):
        print("Session ended.")
        break
    response = handle_request(user_input)
    print(f"\n💬 Response:\n{response}\n")
    print("-" * 60)

🤖 MoE Customer Support Router — Interactive Mode
Type 'exit' or 'quit' to stop.

You: Sourabh  S Hegde PES2UG23CS929

  📨 User Input  : Sourabh  S Hegde PES2UG23CS929
  🔀 Routed To   : [GENERAL EXPERT]

💬 Response:
It seems like you've shared your name and a student ID, Sourabh. Is there something I can help you with or would you like to chat?

------------------------------------------------------------
You: What is the price of Solana?

  📨 User Input  : What is the price of Solana?
  🔀 Routed To   : [TOOL_USE EXPERT]
  🔧 Tool Result : fetch_crypto_price('solana') → {'symbol': 'SOL', 'price_usd': 172.4, 'change_24h': '+4.1%'}

💬 Response:
The current price of Solana (SOL) is $172.40 USD, according to the Mock Market Feed (simulated). Please note that prices update frequently, so this information may change rapidly.

------------------------------------------------------------
You: exit
Session ended.


## Architecture Overview

```
User Query
    │
    ▼
┌──────────────────────────────────┐
│   ROUTER  classify_intent()      │  ← temperature=0, max_tokens=10
└──────────────────┬───────────────┘
                   │ category
    ┌──────────────▼──────────────────────────────┐
    │      ORCHESTRATOR  handle_request()          │
    └──┬──────────┬──────────┬──────────┬──────────┘
       │          │          │          │          │
       ▼          ▼          ▼          ▼          ▼
    Tech       Billing     Sales    Tool Use   General
   Expert      Expert     Expert    Expert     Expert
```

| Expert | Temperature | Purpose |
|--------|-------------|--------|
| Technical | 0.3 | Debugging, code review |
| Billing | 0.5 | Refunds, payment issues |
| Sales | 0.7 | Product info, upgrades |
| Tool Use | 0.2 | Real-time market data |
| General | 0.7 | Fallback / casual chat |

### Key design choices
- **Deterministic routing** (`temperature=0`) avoids flaky classification
- **Narrow `max_tokens` on the router** prevents verbose or ambiguous outputs
- **Fallback guard** in `classify_intent()` ensures unknown outputs never crash the pipeline
- **Injected tool context** keeps the tool-use branch clean — the expert sees pre-fetched data, not a raw tool call